# Working with NBA live data...
Libraries supporting live data do not include support for Pandas. Note: Any call to `{endpoint}.{Class}()` will perform a request. Example: `scoreboard.ScoreBoard()`. In order to avoid multiple requests, set `{endpoint}.{Class}()` to a variable. See sample code below.

## Today's Score Board


In [1]:
# Query nba.live.endpoints.scoreboard and  list games in localTimeZone
from datetime import datetime, timezone
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard

f = "{gameId}: {awayTeam} vs. {homeTeam} @ {gameTimeLTZ}" 

board = scoreboard.ScoreBoard()
print("ScoreBoardDate: " + board.score_board_date)
games = board.games.get_dict()
for game in games:
    gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
    print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))

ScoreBoardDate: 2025-03-10
0022400928: 76ers vs. Hawks @ 2025-03-10 19:30:00-04:00
0022400929: Jazz vs. Celtics @ 2025-03-10 19:30:00-04:00
0022400930: Lakers vs. Nets @ 2025-03-10 19:30:00-04:00
0022400931: Hornets vs. Heat @ 2025-03-10 19:30:00-04:00
0022400932: Wizards vs. Raptors @ 2025-03-10 19:30:00-04:00
0022400933: Pacers vs. Bulls @ 2025-03-10 20:00:00-04:00
0022400934: Magic vs. Rockets @ 2025-03-10 20:00:00-04:00
0022400935: Suns vs. Grizzlies @ 2025-03-10 20:00:00-04:00
0022400936: Nuggets vs. Thunder @ 2025-03-10 20:00:00-04:00
0022400937: Mavericks vs. Spurs @ 2025-03-10 20:30:00-04:00
0022400938: Trail Blazers vs. Warriors @ 2025-03-10 22:00:00-04:00
0022400939: Knicks vs. Kings @ 2025-03-10 22:30:00-04:00


## Box Score

In [2]:
# Get BoxScore
from nba_api.live.nba.endpoints import boxscore
box = boxscore.BoxScore('0022000196') 

In [3]:
# Data Sets
box.game.get_dict()                    #equal to box.get_dict()['game']
#box.arena.get_dict()                  #equal to box.get_dict()['game']['arena']
#box.away_team.get_dict()              #equal to box.get_dict()['game']['awayTeam']
#box.away_team_player_stats.get_dict() #equal to box.get_dict()['game']['awayTeam']['players']
#box.away_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.home_team.get_dict()              #equal to box.get_dict()['game']['homeTeam']
#box.home_team_player_stats.get_dict() #equal to box.get_dict()['game']['homeTeam']['players']
#box.home_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.game_details.get_dict()           #equal to box.get_dict()['game'] scrubbed of all other dictionaries
#box.officials.get_dict()              #equal to box.get_dict()['game']['officials']

{'gameId': '0022000196',
 'gameTimeLocal': '2021-01-17T13:00:00-05:00',
 'gameTimeUTC': '2021-01-17T18:00:00Z',
 'gameTimeHome': '2021-01-17T13:00:00-05:00',
 'gameTimeAway': '2021-01-17T13:00:00-05:00',
 'gameEt': '2021-01-17T13:00:00-05:00',
 'duration': 134,
 'gameCode': '20210117/NYKBOS',
 'gameStatusText': 'Final',
 'gameStatus': 3,
 'regulationPeriods': 4,
 'period': 4,
 'gameClock': 'PT00M00.00S',
 'attendance': 0,
 'sellout': '0',
 'arena': {'arenaId': 17,
  'arenaName': 'TD Garden',
  'arenaCity': 'Boston',
  'arenaState': 'MA',
  'arenaCountry': 'US',
  'arenaTimezone': 'America/New_York'},
 'officials': [{'personId': 2882,
   'name': 'Sean Wright',
   'nameI': 'S. Wright',
   'firstName': 'Sean',
   'familyName': 'Wright',
   'jerseyNum': '4',
   'assignment': 'OFFICIAL1'},
  {'personId': 201638,
   'name': 'Brent Barnaky',
   'nameI': 'B. Barnaky',
   'firstName': 'Brent',
   'familyName': 'Barnaky',
   'jerseyNum': '36',
   'assignment': 'OFFICIAL2'},
  {'personId': 162752

In [4]:
# Getting Box Scores. 
# Note: home_team & away_team have the identicial data structure.
players = box.away_team.get_dict()['players']
f = "{player_id}: {name}: {points} PTS"
for player in players:
    print(f.format(player_id=player['personId'],name=player['name'],points=player['statistics']['points']))

203493: Reggie Bullock: 11 PTS
203944: Julius Randle: 20 PTS
1629011: Mitchell Robinson: 8 PTS
1629628: RJ Barrett: 19 PTS
203901: Elfrid Payton: 9 PTS
203457: Nerlens Noel: 5 PTS
203085: Austin Rivers: 0 PTS
1630193: Immanuel Quickley: 17 PTS
1630167: Obi Toppin: 12 PTS
1628995: Kevin Knox II: 3 PTS
1628372: Dennis Smith Jr.: 1 PTS
1629033: Theo Pinson: 0 PTS
1629649: Ignas Brazdeikis: 0 PTS
201959: Taj Gibson: 0 PTS
1629607: Jared Harper: 0 PTS
202692: Alec Burks: 0 PTS
1628373: Frank Ntilikina: 0 PTS


## Play By Play Data

In [5]:
# Query nba.live.endpoints for the score board of GameID 0022000180 = NYK vs BOS
# Simple PlayByPlay Loop demonstrating data usage
from nba_api.live.nba.endpoints import playbyplay
from nba_api.stats.static import players

pbp = playbyplay.PlayByPlay('0022000196')
line = "{action_number}: {period}:{clock} {player_id} ({action_type})"
actions = pbp.get_dict()['game']['actions'] #plays are referred to in the live data as `actions`
for action in actions:
    player_name = ''
    player = players.find_player_by_id(action['personId'])
    if player is not None:
        player_name = player['full_name']
    print(line.format(action_number=action['actionNumber'],period=action['period'],clock=action['clock'],action_type=action['actionType'],player_id=player_name))

2: 1:PT12M00.00S  (period)
4: 1:PT11M58.00S Grant Williams (jumpball)
7: 1:PT11M47.00S Julius Randle (foul)
9: 1:PT11M47.00S Grant Williams (freethrow)
10: 1:PT11M47.00S  (rebound)
11: 1:PT11M47.00S Grant Williams (freethrow)
12: 1:PT11M45.00S Julius Randle (rebound)
13: 1:PT11M25.00S RJ Barrett (turnover)
14: 1:PT11M25.00S Kemba Walker (steal)
15: 1:PT11M21.00S Kemba Walker (turnover)
16: 1:PT11M07.00S RJ Barrett (2pt)
17: 1:PT11M06.00S Mitchell Robinson (rebound)
18: 1:PT11M06.00S Mitchell Robinson (2pt)
19: 1:PT10M53.00S Tristan Thompson (2pt)
21: 1:PT10M26.00S RJ Barrett (3pt)
22: 1:PT10M24.00S Reggie Bullock Jr. (rebound)


23: 1:PT10M17.00S Julius Randle (3pt)
24: 1:PT10M01.00S Marcus Smart (3pt)
25: 1:PT09M58.00S Elfrid Payton (rebound)
26: 1:PT09M50.00S Elfrid Payton (2pt)
27: 1:PT09M50.00S  (timeout)
28: 1:PT09M35.00S Kemba Walker (3pt)
29: 1:PT09M29.00S RJ Barrett (rebound)
30: 1:PT09M20.00S Reggie Bullock Jr. (3pt)
31: 1:PT09M18.00S Kemba Walker (rebound)
32: 1:PT09M09.00S Grant Williams (2pt)
34: 1:PT08M57.00S Kemba Walker (foul)
36: 1:PT08M43.00S Julius Randle (2pt)
37: 1:PT08M41.00S Grant Williams (rebound)
38: 1:PT08M31.00S Kemba Walker (turnover)
39: 1:PT08M20.00S Reggie Bullock Jr. (3pt)
40: 1:PT08M16.00S Jaylen Brown (rebound)


41: 1:PT08M09.00S Mitchell Robinson (foul)
43: 1:PT07M58.00S Grant Williams (3pt)
44: 1:PT07M56.00S RJ Barrett (rebound)
45: 1:PT07M53.00S Grant Williams (foul)
47: 1:PT07M53.00S Julius Randle (freethrow)
48: 1:PT07M53.00S  (rebound)
49: 1:PT07M53.00S Julius Randle (freethrow)
50: 1:PT07M36.00S Kemba Walker (turnover)
51: 1:PT07M36.00S Julius Randle (steal)
52: 1:PT07M24.00S RJ Barrett (foul)
54: 1:PT07M24.00S RJ Barrett (turnover)
55: 1:PT07M14.00S  (stoppage)
56: 1:PT07M14.00S Kemba Walker (substitution)
57: 1:PT07M14.00S Jeff Teague (substitution)
58: 1:PT07M06.00S Jaylen Brown (3pt)
59: 1:PT07M02.00S Julius Randle (rebound)
60: 1:PT06M52.00S Reggie Bullock Jr. (3pt)
62: 1:PT06M36.00S Marcus Smart (3pt)
63: 1:PT06M33.00S Jaylen Brown (rebound)
64: 1:PT06M31.00S Jaylen Brown (2pt)
65: 1:PT06M31.00S Tristan Thompson (rebound)


66: 1:PT06M27.00S Tristan Thompson (turnover)
67: 1:PT06M27.00S RJ Barrett (steal)
68: 1:PT06M12.00S Elfrid Payton (2pt)
69: 1:PT06M10.00S Mitchell Robinson (rebound)
70: 1:PT06M08.00S Reggie Bullock Jr. (3pt)
71: 1:PT06M05.00S  (rebound)
72: 1:PT06M05.00S  (stoppage)
73: 1:PT05M52.00S Jaylen Brown (3pt)
75: 1:PT05M41.00S Jeff Teague (foul)
77: 1:PT05M41.00S Elfrid Payton (freethrow)
78: 1:PT05M41.00S Grant Williams (substitution)
79: 1:PT05M41.00S Tristan Thompson (substitution)
80: 1:PT05M41.00S Daniel Theis (substitution)
81: 1:PT05M41.00S Semi Ojeleye (substitution)
82: 1:PT05M41.00S Elfrid Payton (freethrow)
83: 1:PT05M17.00S Jeff Teague (2pt)
84: 1:PT05M13.00S RJ Barrett (rebound)


85: 1:PT05M02.00S RJ Barrett (2pt)
86: 1:PT04M43.00S Jeff Teague (3pt)
87: 1:PT04M43.00S Mitchell Robinson (block)
88: 1:PT04M43.00S  (rebound)
89: 1:PT04M43.00S Jeff Teague (foul)
91: 1:PT04M43.00S Mitchell Robinson (substitution)
92: 1:PT04M43.00S Reggie Bullock Jr. (substitution)
93: 1:PT04M43.00S Austin Rivers (substitution)
94: 1:PT04M43.00S Nerlens Noel (substitution)
95: 1:PT04M27.00S Julius Randle (2pt)
96: 1:PT04M26.00S Jeff Teague (rebound)
97: 1:PT04M20.00S Jaylen Brown (3pt)
99: 1:PT04M06.00S Julius Randle (2pt)
101: 1:PT03M54.00S Jaylen Brown (3pt)
103: 1:PT03M32.00S RJ Barrett (2pt)
104: 1:PT03M31.00S RJ Barrett (rebound)
105: 1:PT03M31.00S Jaylen Brown (foul)
107: 1:PT03M31.00S RJ Barrett (freethrow)
108: 1:PT03M31.00S Marcus Smart (substitution)
109: 1:PT03M31.00S Payton Pritchard (substitution)


...

598: 4:PT06M33.00S Jaylen Brown (substitution)
599: 4:PT06M33.00S Tristan Thompson (substitution)
600: 4:PT06M33.00S Marcus Smart (substitution)
601: 4:PT06M33.00S Aaron Nesmith (substitution)
602: 4:PT06M33.00S Javonte Green (substitution)
603: 4:PT06M33.00S Tremont Waters (substitution)
604: 4:PT06M26.00S Julius Randle (3pt)
605: 4:PT06M20.00S Mitchell Robinson (rebound)
606: 4:PT06M20.00S Mitchell Robinson (2pt)
607: 4:PT06M10.00S RJ Barrett (foul)
609: 4:PT06M10.00S Javonte Green (freethrow)
610: 4:PT06M10.00S Javonte Green (freethrow)
611: 4:PT05M49.00S Immanuel Quickley (3pt)
612: 4:PT05M39.00S Tremont Waters (3pt)
613: 4:PT05M37.00S Javonte Green (rebound)
614: 4:PT05M34.00S Aaron Nesmith (2pt)
615: 4:PT05M34.00S Immanuel Quickley (block)
616: 4:PT05M33.00S  (rebound)
617: 4:PT05M33.00S  (stoppage)


618: 4:PT05M22.00S RJ Barrett (2pt)
620: 4:PT05M12.00S Javonte Green (3pt)
621: 4:PT05M10.00S Julius Randle (rebound)
622: 4:PT04M59.00S Javonte Green (foul)
624: 4:PT04M59.00S Jeff Teague (substitution)
625: 4:PT04M59.00S Semi Ojeleye (substitution)
626: 4:PT04M53.00S Grant Williams (foul)
628: 4:PT04M53.00S RJ Barrett (freethrow)
629: 4:PT04M53.00S Mitchell Robinson (substitution)
630: 4:PT04M53.00S Julius Randle (substitution)
631: 4:PT04M53.00S Obi Toppin (substitution)
632: 4:PT04M53.00S Nerlens Noel (substitution)
633: 4:PT04M53.00S RJ Barrett (freethrow)
634: 4:PT04M41.00S Grant Williams (3pt)
635: 4:PT04M38.00S RJ Barrett (rebound)
636: 4:PT04M16.00S Immanuel Quickley (2pt)
637: 4:PT04M05.00S Semi Ojeleye (3pt)
638: 4:PT04M02.00S Obi Toppin (rebound)
639: 4:PT03M53.00S Reggie Bullock Jr. (3pt)


641: 4:PT03M44.00S Immanuel Quickley (foul)
643: 4:PT03M44.00S Semi Ojeleye (freethrow)
644: 4:PT03M44.00S Immanuel Quickley (substitution)
645: 4:PT03M44.00S RJ Barrett (substitution)
646: 4:PT03M44.00S Dennis Smith Jr. (substitution)
647: 4:PT03M44.00S Kevin Knox II (substitution)
648: 4:PT03M44.00S Semi Ojeleye (freethrow)
649: 4:PT03M31.00S Grant Williams (foul)
651: 4:PT03M31.00S Nerlens Noel (freethrow)
652: 4:PT03M31.00S  (rebound)
653: 4:PT03M31.00S Reggie Bullock Jr. (substitution)
654: 4:PT03M31.00S Theo Pinson (substitution)
655: 4:PT03M31.00S Nerlens Noel (freethrow)
656: 4:PT03M21.00S Aaron Nesmith (3pt)
657: 4:PT03M19.00S Semi Ojeleye (rebound)
658: 4:PT03M17.00S Theo Pinson (foul)
660: 4:PT03M17.00S Javonte Green (freethrow)
661: 4:PT03M17.00S Kevin Knox II (substitution)
662: 4:PT03M17.00S Ignas Brazdeikis (substitution)


663: 4:PT03M17.00S Javonte Green (freethrow)
664: 4:PT03M17.00S Obi Toppin (rebound)
665: 4:PT03M00.00S Theo Pinson (3pt)
666: 4:PT02M57.00S Aaron Nesmith (rebound)
667: 4:PT02M41.00S Javonte Green (2pt)
669: 4:PT02M27.00S Dennis Smith Jr. (2pt)
670: 4:PT02M24.00S Tremont Waters (rebound)
671: 4:PT02M20.00S Theo Pinson (foul)
673: 4:PT02M20.00S Semi Ojeleye (freethrow)
674: 4:PT02M20.00S Semi Ojeleye (freethrow)
675: 4:PT02M04.00S Nerlens Noel (2pt)
676: 4:PT02M01.00S Javonte Green (rebound)
677: 4:PT01M54.00S Grant Williams (3pt)
678: 4:PT01M52.00S Nerlens Noel (rebound)
679: 4:PT01M38.00S Aaron Nesmith (foul)
681: 4:PT01M38.00S Dennis Smith Jr. (freethrow)
682: 4:PT01M38.00S Dennis Smith Jr. (freethrow)
683: 4:PT01M38.00S Semi Ojeleye (rebound)


684: 4:PT01M22.00S Aaron Nesmith (2pt)
685: 4:PT01M22.00S Nerlens Noel (block)
686: 4:PT01M19.00S Obi Toppin (rebound)
687: 4:PT01M15.00S Ignas Brazdeikis (turnover)
688: 4:PT01M15.00S Tremont Waters (steal)
689: 4:PT01M13.00S Javonte Green (turnover)
690: 4:PT01M13.00S Obi Toppin (steal)
691: 4:PT01M12.00S  (stoppage)
692: 4:PT00M56.90S Dennis Smith Jr. (3pt)
693: 4:PT00M55.20S  (rebound)
694: 4:PT00M55.20S  (stoppage)
695: 4:PT00M49.70S Tremont Waters (3pt)
696: 4:PT00M35.90S Nerlens Noel (2pt)
697: 4:PT00M21.20S Nerlens Noel (foul)
699: 4:PT00M21.20S Grant Williams (freethrow)
701: 4:PT00M21.20S Grant Williams (freethrow)
702: 4:PT00M19.70S Nerlens Noel (rebound)
703: 4:PT00M00.00S  (period)
704: 4:PT00M00.00S  (game)


# Bettings Odds Data

In [6]:
from nba_api.live.nba.endpoints import Odds
import json

# Fetch odds data
odds = Odds()
games_list = odds.get_games().get_dict()

# Print first 2 games with nice formatting
print(json.dumps(games_list[:3], indent=2))

[
  {
    "gameId": "0022400913",
    "sr_id": "sr:match:52632103",
    "srMatchId": "52632103",
    "homeTeamId": "1610612745",
    "awayTeamId": "1610612740",
    "markets": [
      {
        "name": "2way",
        "odds_type_id": 1,
        "group_name": "regular",
        "books": [
          {
            "id": "sr:book:108",
            "name": "Sportsbet",
            "outcomes": [
              {
                "odds_field_id": 1,
                "type": "home",
                "odds": "1.370",
                "opening_odds": "1.360",
                "odds_trend": "up"
              },
              {
                "odds_field_id": 2,
                "type": "away",
                "odds": "3.220",
                "opening_odds": "3.300",
                "odds_trend": "down"
              }
            ],
            "url": "https://ad.doubleclick.net/ddm/clk/594881279;402977564;g;gdpr=${GDPR};gdpr_consent=${GDPR_CONSENT_755}",
            "countryCode": "AUS"
          },


You can further analyze by using other functionality available as a part of this library to extract meaning from the raw data. In example:

In [7]:
from nba_api.live.nba.endpoints import Odds
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder

# Fetch odds data for today's NBA games
odds = Odds()
games_list = odds.get_games().get_dict()

# Get first game only
game = games_list[0]
game_id = game.get('gameId', 'Unknown')
home_team = teams._find_team_name_by_id(game['homeTeamId'])['full_name'] # Use teams module to get team names
away_team = teams._find_team_name_by_id(game['awayTeamId'])['full_name']

# Get game date using LeagueGameFinder
gamefinder = leaguegamefinder.LeagueGameFinder(
    league_id_nullable="00",
    game_id_nullable=game_id
)
game_info = gamefinder.get_data_frames()[0]
specific_game = game_info[game_info['GAME_ID'] == game_id]
game_date = specific_game['GAME_DATE'].iloc[0]

# Display basic game information
print(f"Game ID: {game_id}")
print(f"Game Date: {game_date}")
print(f"Home Team: {home_team}")
print(f"Away Team: {away_team}")

# Display 2-way odds
two_way_market = next((m for m in game['markets'] if m['name'] == '2way'), None)
if two_way_market and two_way_market['books']:
    first_book = two_way_market['books'][0]
    print(f"\n2-way Odds from {first_book['name']}:")
    for outcome in first_book['outcomes']:
        team_type = outcome['type']
        team_name = home_team if team_type == 'home' else away_team
        odds = outcome['odds']
        print(f"  {team_type.capitalize()} ({team_name}): {odds}")
        trend = outcome['odds_trend']
        print(f"  Odds trend is: {trend}")
        opening = outcome['opening_odds']
        print(f"  Opening odds were: {opening}")

# Display spread odds
spread_market = next((m for m in game['markets'] if m['name'] == 'spread'), None)
if spread_market and spread_market['books']:
    first_book = spread_market['books'][0]
    print(f"\nSpread Odds from {first_book['name']}:")
    for outcome in first_book['outcomes']:
        team_type = outcome['type']
        team_name = home_team if team_type == 'home' else away_team
        spread = outcome.get('spread', 'N/A')
        odds = outcome['odds']
        print(f"  {team_type.capitalize()} ({team_name}) with spread {spread}: {odds}")
        trend = outcome['odds_trend']
        print(f"  Odds trend is: {trend}")
        opening = outcome['opening_odds']
        print(f"  Opening odds: {opening}")
        opening_spread = outcome['opening_spread']
        print(f"  Opening spread: {opening_spread}")

Game ID: 0022400913
Game Date: 2025-03-08
Home Team: Houston Rockets
Away Team: New Orleans Pelicans

2-way Odds from Sportsbet:
  Home (Houston Rockets): 1.370
  Odds trend is: up
  Opening odds were: 1.360
  Away (New Orleans Pelicans): 3.220
  Odds trend is: down
  Opening odds were: 3.300

Spread Odds from Sportsbet:
  Home (Houston Rockets) with spread -6.5: 1.880
  Odds trend is: up
  Opening odds: 1.880
  Opening spread: -7.5
  Away (New Orleans Pelicans) with spread 6.5: 1.920
  Odds trend is: down
  Opening odds: 1.920
  Opening spread: 7.5
